In [41]:
# pip install transformers datasets scikit-learn

In [42]:
from transformers import DistilBertTokenizer

# Tokenizador
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
inputs = tokenizer(["Necesito mi póliza", "Hola"], padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101, 26785,  2229,  9956,  2771, 14955, 21335,   102],
        [  101,  7570,  2721,   102,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0]])}


In [43]:
from datasets import load_dataset

# Cargar el dataset desde el archivo CSV
dataset = load_dataset("csv", data_files="dataset_intenciones.csv")

In [44]:
# Dividir el dataset en entrenamiento y prueba (80/20)
dataset = dataset["train"].train_test_split(test_size=0.2)

# Mapeo de etiquetas a números enteros (para 'intent')
intent_to_id = {
    "saludo": 0,
    "obtener_poliza": 1,
    "consultar_cobertura": 2,
    "cancelar_seguro": 3,
    "renovar_seguro": 4,
    "modificar_datos_personales": 5,
    "informar_accidente": 6,
    "consultar_estado_pago": 7,
    "preguntar_precio": 8,
    "reclamo": 9,
}

# Convertir las etiquetas a enteros
dataset = dataset.map(lambda x: {'labels': intent_to_id[x['intent']]}, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]


TypeError: unhashable type: 'list'

In [ ]:
# Tokenización
def tokenize(batch):
    # Tokenizar el texto
    tokenized_batch = tokenizer(batch["text"], padding=True, truncation=True)
    # No es necesario renombrar ya que 'intent' ya se ha renombrado a 'labels'
    return tokenized_batch

# Aplicar la tokenización al dataset
dataset = dataset.map(tokenize, batched=True)

Map: 100%|██████████| 10/10 [00:00<00:00, 1428.92 examples/s]


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Cargar el modelo preentrenado para clasificación
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import transformers
print(transformers.__version__)


4.51.3


In [ ]:
# Configuración de los parámetros de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    eval_steps=500,  # Evaluar cada 500 pasos
    logging_steps=100,  # Imprimir la pérdida cada 100 pasos
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    save_strategy="epoch"  # Guardar después de cada época
)

# Configuración del entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer
)

# Iniciar el entrenamiento
trainer.train()



C:\Users\luisa\AppData\Local\Temp\ipykernel_25984\2634922214.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
trainer.evaluate()


In [ ]:
model.save_pretrained("modelo_agente_seguro")
tokenizer.save_pretrained("modelo_agente_seguro")


In [51]:
from transformers import DistilBertTokenizer
from datasets import load_dataset

# Tokenizador
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Cargar el dataset desde el archivo CSV
dataset = load_dataset("csv", data_files="dataset_intenciones.csv")

# Dividir el dataset en entrenamiento y prueba (80/20)
dataset = dataset["train"].train_test_split(test_size=0.2)

# Mapeo de etiquetas a números enteros (para 'intent')
intent_to_id = {
    "saludo": 0,
    "obtener_poliza": 1,
    "consultar_cobertura": 2,
    "cancelar_seguro": 3,
    "renovar_seguro": 4,
    "modificar_datos_personales": 5,
    "informar_accidente": 6,
    "consultar_estado_pago": 7,
    "preguntar_precio": 8,
    "reclamo": 9,
}

# Asegurarnos de que 'intent' esté en el formato adecuado (str) y aplicar el mapeo de etiquetas
def map_intents(batch):
    # Convertir la columna 'intent' en un listado de índices numéricos
    return {'labels': [intent_to_id[x] for x in batch['intent']]}

# Aplicamos la función map_intents a todo el dataset
dataset = dataset.map(map_intents, batched=True)

# Tokenización
def tokenize(batch):
    # Tokenizar el texto
    tokenized_batch = tokenizer(batch["text"], padding=True, truncation=True)
    return tokenized_batch

# Aplicar la tokenización al dataset
dataset = dataset.map(tokenize, batched=True)

from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Cargar el modelo preentrenado para clasificación
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(intent_to_id))

# Configuración de los parámetros de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    eval_steps=500,  # Evaluar cada 500 pasos
    logging_steps=100,  # Imprimir la pérdida cada 100 pasos
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    save_strategy="epoch"  # Guardar después de cada época
)

# Configuración del entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer
)

# Iniciar el entrenamiento
trainer.train()

trainer.save_model("./results")


Map: 100%|██████████| 10/10 [00:00<00:00, 1429.16 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\luisa\AppData\Local\Temp\ipykernel_25984\366377895.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


In [53]:
from transformers import DistilBertTokenizer
import torch

# Tokenizador y modelo cargados previamente
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("./results")  # Ruta al modelo entrenado

# Función para predecir la intención
def predict_intent(text):
    # Tokenizar el texto
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512)
    
    # Asegurarse de que el modelo está en modo evaluación
    model.eval()
    
    # Hacer la predicción (sin gradientes)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Obtener la predicción (índice de la clase con mayor probabilidad)
    logits = outputs.logits
    predicted_class_idx = torch.argmax(logits, dim=-1).item()

    # Convertir el índice en la etiqueta correspondiente
    id_to_intent = {v: k for k, v in intent_to_id.items()}
    predicted_intent = id_to_intent[predicted_class_idx]
    
    return predicted_intent

# Probar con algunos ejemplos de texto real
text = ["¿Mi pago de seguro ha sido procesado?", "¿Dónde está mi póliza?", "Quiero hacer un reclamo"]

for t in text:
    print(f"Texto: {t} => Predicción: {predict_intent(t)}")


Texto: ¿Mi pago de seguro ha sido procesado? => Predicción: preguntar_precio
Texto: ¿Dónde está mi póliza? => Predicción: preguntar_precio
Texto: Quiero hacer un reclamo => Predicción: informar_accidente
